In [1]:
%cd ../../

/home/shpotes/Courses/EAFIT/inteligencia-artificial/santiago/clustering/clustering


In [2]:
import pathlib
import pandas as pd
import numpy as np
import optuna
from src import Substractive, KMeansPlusPlus
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score, silhouette_score

In [3]:
data_dir = pathlib.Path('data/retuers')
news = pd.read_csv(data_dir / 'raw.csv')
w2v_md = np.load(data_dir / 'en-core-md-representations.npy')

In [4]:
r_a = 2 #trail.suggest_uniform('r_a', 1e-2, 1e2)

sub = Substractive(r_a=r_a, tol=0.01)
labels = sub.fit_predict(w2v_md)

/home/shpotes/.cache/pypoetry/virtualenvs/clustering-tBf8LVkM-py3.8/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [5]:
def objective(trail):
    r_a = trail.suggest_loguniform('r_a', 1e-1, 1e1)
    sub = Substractive(r_a=r_a, tol=0.01)
    labels = sub.fit_predict(w2v_md)
    
    if len(np.unique(labels)) == 1:
        return -2

    return davies_bouldin_score(w2v_md, labels)

In [6]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2020-09-20 19:06:04,239] A new study created in memory with name: no-name-706afc99-7b9f-4c1f-ab20-ff5d070399f2
[I 2020-09-20 19:06:06,565] Trial 0 finished with value: -2.0 and parameters: {'r_a': 0.5646187481058749}. Best is trial 0 with value: -2.0.
[I 2020-09-20 19:06:08,823] Trial 1 finished with value: -2.0 and parameters: {'r_a': 5.1165110911194525}. Best is trial 0 with value: -2.0.
[I 2020-09-20 19:06:10,838] Trial 2 finished with value: -2.0 and parameters: {'r_a': 0.6833049007955353}. Best is trial 0 with value: -2.0.
[I 2020-09-20 19:06:12,926] Trial 3 finished with value: -2.0 and parameters: {'r_a': 0.17762851524757003}. Best is trial 0 with value: -2.0.
[I 2020-09-20 19:06:14,971] Trial 4 finished with value: -2.0 and parameters: {'r_a': 0.723432868080692}. Best is trial 0 with value: -2.0.
[I 2020-09-20 19:06:17,040] Trial 5 finished with value: -2.0 and parameters: {'r_a': 3.1003973678023287}. Best is trial 0 with value: -2.0.
[I 2020-09-20 19:06:19,004] Trial 6 fini